# OpenCVでとことん画像処理 - 後半
このノートは **後半** のノートになります。後半では

- [空間フィルタリング](#spatial)
	- [平滑化](#smoothing)
		- [平均化フィルタ](#mean)
		- [加重平均化(ガウシアン)フィルタ](#weight_mean)
		- [メディアンフィルタ](#median)
	- [エッジ抽出](#edge)
		- [Cannyアルゴリズムによるエッジ検出](#canny)
	- [鮮鋭化(アンシャープマスキング)](#sharpening)
	- [エッジを保存した平滑化](#edge_smoothing)
		- [バイラテラルフィルタ](#bilateral)
		- [ノンローカルミーンフィルタ](#non_local_mean)
- [二値化](#binarize)
	- [単純なしきい値処理](#threshold)
	- [複雑なしきい値処理](#complex_threshold)
- [領域分割](#area_separate)
    - [クロマキーによるマスク処理](#chromakey)
        - [画像合成](#composition)
    - [ミーンシフト法](#mean_shift)
    - [grabcut法](#grabcut)
- [まとめ](#conclusion)
    
を扱います。

※追記: Python 3.8.2, chromium ベースのブラウザで動作確認してあります。

## モジュールのインポート

In [ ]:
# これを書くとjupyterでmatplotlibなどが出力する画像の解像度が上がる
# %config InlineBackend.figure_format = 'retina'

import cv2
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import sys
sys.path.append("../../")
import time

from utils.plot import compare_plot

print(cv2.__version__)
print(matplotlib.__version__)

In [ ]:
from tqdm import tqdm_notebook as tqdm

<a id="basic"></a>
## 画像の読み込み

In [ ]:
img = cv2.cvtColor(cv2.imread("../../data/Lenna_flat.png"), cv2.COLOR_BGR2RGB)
img_cat = cv2.cvtColor(cv2.imread("../../data/kuroneko.png"), cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img)

In [ ]:
# 上のRGB画像をグレー画像にして読み込みます
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
plt.imshow(img_gray, cmap="gray")

<a id="spatial"></a>
## 空間フィルタリング
前半では画素ごとの濃淡変換を対応する**画素値**のみで行っていたが、画素値のみではなくその周辺の画素も含めた領域内の画素値を元に計算します。この処理を**空間フィルタリング**と呼び、そこで用いるフィルタは一般的に空間フィルタと呼びます。

<a id="smoothing"></a>
## 平滑化
画像になめらかな濃淡変換を与える処理を**平滑化**と呼んでおり、画像に含まれるノイズなどの不要な濃淡変動を軽減するときなどに用いられます。

<a id="mean"></a>
### 平均化フィルタ
画像をぼかすためにフィルタのサイズを元に平均をとったフィルタを用いて画像をぼかす手法を**平均化フィルタ**と呼びます。

<img src="../../fig/mean_filter.png" width=400>

### 手動でフィルタを定義する場合

In [ ]:
# フィルタの定義
size = 5
kernel = np.ones((size,size),np.float32) / size**2

In [ ]:
kernel

In [ ]:
img_mean_sm = cv2.filter2D(img, -1, kernel)

### 平均化フィルタを関数で実行する場合

In [ ]:
img_blur = cv2.blur(img, (size, size))

In [ ]:
compare_plot([img, img_blur], ["before", "after"])

<a id="weight_mean"></a>
### 加重平均化(ガウシアンフィルタ)
単純なフィルターサイズに基づいた平均値ではなく、中央に近いほど大きな重みをつける**加重平均化フィルタ**というものもあります。その重みの中でも**ガウス分布(標準正規分布)**に近づけたものを**ガウシアンフィルタ**と呼びます。

標準正規分布を2次元(画像)に拡張した分布は以下のように表されます。

$$ f(x) =  \frac{1}{ 2 \pi \sigma^2 } \exp \left( -  \frac{x^2 + y^2}{2 \sigma^2} \right) $$

<table border="0">
<tr>
<td><img src="../../fig/two_dim_gausian.png" width=300></td>
<td><img src="../../fig/weight_mean_filter.png" width=300></td>
</tr>
</table>

In [ ]:
img_g_blur = cv2.GaussianBlur(img,(5,5),0)

In [ ]:
compare_plot([img, img_g_blur], ["before", "after"])

単純な平均化フィルタに比べて見た目にあまり大きな違いはありませんが、よりなめらかで自然な平滑化の効果を期待することができます。

<a id="median"></a>
### メディアンフィルタ
メディアンフィルタはフィルタサイズで指定された範囲の中の中央値を元の値と入れ替えるというもので、特にソルト・ペッパーノイズ(インパルスノイズ)の除去に強いと言われている手法です。

In [ ]:
# 前半で作成したソルト・ペッパーノイズ画像を読み込む
img_sp = cv2.imread("../../data/img_sp.png")

In [ ]:
size = 5
img_blur = cv2.blur(img_sp, (size, size))
img_g_blur = cv2.GaussianBlur(img_sp, (size,size), 0)
img_m_blur = cv2.medianBlur(img_sp, size)

In [ ]:
compare_plot([img_sp, img_blur, img_g_blur, img_m_blur], \
            ["salt&pepper img", "mean_filter", "gaussian_mean_filter", "median_filter"])

<a id="edge"></a>
## エッジ抽出
画像中で明るさが急激に変化する**エッジ部分**を取り出す手法をエッジ抽出と呼びます。これらの処理は画像中の特徴や図形を検出するための前処理として用いられることが多いです。

またエッジ抽出には微分フィルタやソーベルフィルタ、ラプラシアンフィルタなど様々な手法が存在しますが、その中でも輪郭の検出漏れや誤検出が少ない手法であ**Cannyアルゴリズム**について説明します。

<a id="canny"></a>
### Cannyアルゴリズムによるエッジ抽出
Cannyアルゴリズムでは大まかな流れとして、

1. ノイズの除去
2. 輪郭の抽出
3. 非極大抑制
4. ヒステリシスしきい値処理

の4つで処理を行っていきます。詳しい処理についてはここでは時間の都合で説明を割愛させていただきますが、わかりやすく詳しい解説をノート最下部の参考サイトに乗せていますのでそちらを参照ください。

In [ ]:
img_canny = cv2.Canny(img_gray,100,200) #引数は 画像,最小閾値,最大閾値

In [ ]:
compare_plot([img_gray, img_canny], ["before", "after"], ["gray", "gray"])

<a id="sharpening"></a>
## 鮮鋭化
エッジ抽出はエッジのみの抽出でしたが、鮮鋭化という手法では元の画像の濃淡を残したままエッジを強調します。元の画像に対して平滑化処理をした上で、平滑化した画像から元の画像を引くことでエッジが得られ、この差の画像を元画像に足し合わせることで、エッジが強調された画像を得ることができます。このような処理を**アンシャープマスキング**と呼んでいます。

In [ ]:
img_cp1 = img.copy()

In [ ]:
def unsharp_masking(img, i):
    img_gblur = cv2.GaussianBlur(img, (i,i), 10.0)
    unsharp_image = cv2.addWeighted(img, 1.5, img_gblur, -0.5, 0, img)
    return unsharp_image

In [ ]:
# 複数回実行で重ねがけできます
compare_plot([img, unsharp_masking(img_cp1, 9)], \
             ["img", "img + edge"])

In [ ]:
img_cp2 = img_cat.copy()

In [ ]:
compare_plot([img_cat, unsharp_masking(img_cp2, 9)], \
             ["img", "img + edge"])

エッジが強調され、より鮮鋭化の度合いが増していることが比較してわかります。

<a id="edge_smoothing"></a>
## エッジを保存した平滑化
これまで紹介した単純な平滑化では、画像に含まれるノイズなどの濃淡変動を軽減できていますが、それと同時に画像にもともとあるエッジもなめらかにしてしまうという問題がありました。そこで平滑化と鮮鋭化の2つを満たすような平滑化の手法が提案されており、今回はその中でも**バイラテラルフィルタ**と**ノンローカルミーンフィルタ**の2つを紹介していきます。

<a id="birateral"></a>
### バイラテラルフィルタ
平滑化アルゴリズムの一つであるガウシアンフィルタでは注目画素と周辺がその距離にガウス分布で近似した重みをかけることで平滑化していましたが、バイラテラルフィルタではそれに加えて**注目画素と周辺画素の画素値の差**についてもガウス分布で重み付けをしています。

大雑把に説明すると、カーネルの中心の輝度値と差の少ないところだけをガウシアンフィルタで平滑化することで、近い色はぼかして、遠い色は際立たせることができます。輪郭付近の重みを0か1にするかは正規分布で決めていくことで差が小さいと重みが大きく、輝度差が大きいと重みが小さくなるように、なだらかに変化します。

<img src="../../fig/birateral.png">

In [ ]:
img_cp1 = img.copy()

In [ ]:
def birateral_filter(img, n):
    """
    オプションについて
    
    cv2.bilateralFilter(img, d, sigmaColor, sigmaSpace, 
                                dst, borderType)
    
    img ： 入力画像
    d ： 注目画素をぼかすために使う領域
    sigmaColor： 色についての標準偏差(大きいと画素の差が大きくても大きな重みを使う)
    sigmaSpace：色についての標準偏差(大きいと画素の差が大きくても大きな重みを使う)
    """
    for _ in range(n):
        img = cv2.bilateralFilter(img, 9, 75, 75)
    return img

In [ ]:
# バイラテラルフィルタを複数かけて比較
compare_plot([img] + [birateral_filter(img_cp1, i) for i in range(1,5)], \
             ["before"] + ["birateral_{}".format(i) for i in range(1,5)])

<a id="non_local_mean"></a>
### ノンローカルミーンフィルタ
ノンローカルミーンフィルタはテンプレートマッチングのように周辺画素を含めた領域が、注目がその周辺領域にどれだけ似ているかで重みを決めていきます。

<img src="../../fig/nlmf.png" width=400>

In [ ]:
def non_local_mean_filter(img, n):
    """
    オプションについて
    
    cv2.fastNlMeansDenoisingColored(img, dst, h, hColor, 
                                templateWindowSize, searchWindowSize)
    
    img ： 入力画像
    dst ： 出力画像
    h ： 輝度成分のフィルタの平滑化の度合い
    hColor： 色成分の不フィルタの平滑化の度合い
    templateWindowSize：周辺領域のテンプレートサイズ
    searchWindowSize：重みを探索する領域サイズ
    """
    for _ in range(n):
        img = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    return img

In [ ]:
img_cp1 = img.copy()

In [ ]:
# ノンローカルミーンフィルタを複数かけて比較
compare_plot([img] + [non_local_mean_filter(img_cp1, i) for i in range(1,5)], \
             ["before"] + ["non_local_mean_filter_{}".format(i) for i in range(1,5)])

上記の2つに示されるように、画像をぼやかしながらなおかつエッジが残ったシャープな画像を作る際にバイラテラルフィルタとノンローカルミーンフィルタは最適な処理になっています。

<a id="binarize"></a>
## 2値化
前半で少しだけ2値化について触れましたが、2値化して得たグレースケール画像を用いた画像処理は画像解析において多くの成果を上げてきています。その2値化処理についてより深く掘り下げていきます。

<a id="threshold"></a>
## 単純なしきい値処理
2値画像では、ある画素以上の画素値を1、それ未満の値を0としたときにその境界の値を**しきい値**と呼び、文字認識を行うような場合に必要となる処理です。

OpenCVでは2値化する形式を複数選ぶことができ、それらを形式ごとにどういう処理になっているかを確認していきたいと思います

```python
"""
img : 画像
thresh : しきい値
max_value : 画素の最大値
2値化するためのタイプ : 後述
"""

# OpenCVでの2値化に使用する関数
cv2.threshold(img, thresh, max_value, type)
```

+ THRESH_BINARY
    + しきい値を元に白と黒に2値化します
    
$$ {\displaystyle dst(x, y) = \left\{ \begin{array}{l} maxValue & (src(x, y) > threshold) \\ 0 & (otherwise) \end{array} \right. } $$

+ THRESH_BINARY_INV
    + THRESH_BINARYを逆にしたものです
    
$$ {\displaystyle dst(x, y) = \left\{ \begin{array}{l} 0 & (src(x, y) > threshold) \\ maxValue & (otherwise) \end{array} \right. } $$

+ THRESH_TRUNC
    + しきい値を超えていればしきい値を画素値にし、そうでない場合はそのままです
    
$$ {\displaystyle dst(x, y) = \left\{ \begin{array}{l} threshhold & (src(x, y) > threshold) \\ src(x,y) & (otherwise) \end{array} \right. } $$

+ THRESH_TOZERO
    + しきい値を超えていればそのままで、そうでない値を黒にします

$$ {\displaystyle dst(x, y) = \left\{ \begin{array}{l} src(x,y) & (src(x, y) > threshold) \\ 0 & (otherwise) \end{array} \right. } $$

+ THRESH_TOZERO_INV
    + THRESH_TOZERO_INVを逆にしたものです

$$ {\displaystyle dst(x, y) = \left\{ \begin{array}{l} 0 & (src(x, y) > threshold) \\ src(x,y) & (otherwise) \end{array} \right. } $$

In [ ]:
_,thresh1 = cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY)
_,thresh2 = cv2.threshold(img_gray,127,255,cv2.THRESH_BINARY_INV)
_,thresh3 = cv2.threshold(img_gray,127,255,cv2.THRESH_TRUNC)
_,thresh4 = cv2.threshold(img_gray,127,255,cv2.THRESH_TOZERO)
_,thresh5 = cv2.threshold(img_gray,127,255,cv2.THRESH_TOZERO_INV)
_,thresh6 = cv2.threshold(img_gray,0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)

In [ ]:
compare_plot([img_gray, thresh1, thresh2, thresh3, thresh4, thresh5, thresh6], \
            ["gray", "binary", "binary_inv", "trunc", "to_zero", "to_zero_inv", "otsu"],
            ["gray" for _ in range(7)])

<a id="complex_threshold"></a>
## 複雑なしきい値処理
これまでの単純な2値化と違い、様々な条件下において2値化処理を行っていきます。

<a id="adaptive"></a>
### 適応的しきい値処理
画像の光源環境によって明るさが異なるような画像を2値化する際に**適応的しきい値処理**を行います。適応的しきい値処理では、画像中の小領域ごとにしきい値を計算するため、単純なしきい値処理よりも良い結果を得られるのが特徴です。

+ cv2.adaptiveThreshold(img, max_value, adaptive_thresh, threshhold, blocksize, c)
    + adaptive_thresh
        + 小領域内のしきい値の計算方法
            + ADAPTIVE_THRESH_MEAN_C:近傍領域の中央値をしきい値
            + ADAPTIVE_THRESH_GAUSSIAN_C:近傍領域の重み付け平均値をしきい値
    + blocksize
        + しきい値計算に使用する近傍領域のサイズ(blocksize > 1)
    + c
        + 計算されたしきい値から引く定数

In [ ]:
# 適応的しきい値処理をするために一度メディアンフィルタをかけます
img_med = cv2.medianBlur(img_gray, 5)

In [ ]:
adap1 = cv2.adaptiveThreshold(img_med, 255, cv2.ADAPTIVE_THRESH_MEAN_C, \
                            cv2.THRESH_BINARY, 11, 2)
adap2 = cv2.adaptiveThreshold(img_med, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, \
                            cv2.THRESH_BINARY, 11, 2)

In [ ]:
compare_plot([img_med, thresh1, adap1, adap2], \
            ["median", "thresh_binary", "thresh_mean", "thresh_gaussian"], \
            ["gray" for _ in range(4)])

<a id="area_separate"></a>
## 領域分割
画像に写っている対象を構成する領域を取り出して解析したい場合に行う処理が**領域分割**になります。そのなかでも代表的な手法を中心に紹介していきたいと思います。

<a id="chromakey"></a>
### クロマキーによるマスク処理
クロマキーとはある特定の色を選択してそれを背景とみなすことで、領域分割を行う手法です。主にクロマキー処理では領域分割後に画像を合成するような処理に応用されます。任意の2枚の画像を合成するまでを一貫してやっていきましょう。

In [ ]:
img_chroma = cv2.cvtColor(cv2.imread("../../data/kuroneko_chromakey.png"), cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(img_chroma)

In [ ]:
# 今回は画素値[0,0,255]を対象にクロマキーで分割していく
img_chroma[0][0]

In [ ]:
# クロマキーで抽出する画素をRGBごとに照らし合わせます
img_mask = (img_chroma == [0,0,255])
img_mask = img_mask.astype('uint8') * 255

In [ ]:
# マスクをグレースケールにして2値化してmedianfilterをかけて粗を取ります
img_mask_g1 = cv2.cvtColor(img_mask, cv2.COLOR_RGB2GRAY)
img_mask_g2 = cv2.threshold(img_mask_g1, 254, 255, cv2.THRESH_BINARY_INV)[1]
img_mask_g = cv2.medianBlur(img_mask_g2, 5)

In [ ]:
img_chromakey = cv2.bitwise_and(img_chroma, cv2.cvtColor(img_mask_g, cv2.COLOR_GRAY2RGB))

In [ ]:
compare_plot([img_chroma, img_mask, img_mask_g1, img_mask_g2, img_mask_g, img_chromakey], \
            ["original", "mask(RGB)", "mask(Gray)", "mask(threshold)", "mask(medianBlur)", "chromakey"], \
            [None, None, "gray", "gray", "gray", None])

<a id="composition"></a>
### 画像合成

In [ ]:
img_pydata = cv2.cvtColor(cv2.imread("../../data/pydataokinawa.png"), cv2.COLOR_BGR2RGB)
img_pydata = cv2.resize(img_pydata, img_chromakey.shape[:2])

In [ ]:
# pydata画像からcat画像をくり抜きます
img_maskn = cv2.bitwise_not(img_mask_g)
img_pydata_ext = cv2.bitwise_and(img_pydata, cv2.cvtColor(img_maskn, cv2.COLOR_GRAY2RGB))

In [ ]:
img_dst = cv2.bitwise_or(img_pydata_ext, img_chromakey)

In [ ]:
compare_plot([img_pydata, img_chromakey, img_pydata_ext, img_dst], \
            ["original", "chromakey", "extract", "composition"])

<a id="mean_shift"></a>
### ミーンシフト法
ミーンシフト法は画像領域の分割や対象画像の追跡に用いられる手法です。ここでは色の減算にミーンシフトを用いることで領域を分割しやすくしています。

ミーンシフトではある画素に対して、その点の中心となる半径の円を考えて、その重心を求めていきます。その重心を中心にという形で重心の極大値を求めていくと似た色が集まってくる(領域が分割される)仕組みになっています。

<img src="../../fig/meanshift.png">

In [ ]:
"""
cv2.pyrMeanShiftFiltering(img, sp, sr)

sp:円の半径
sr:画素値の範囲
"""

img_ms = cv2.pyrMeanShiftFiltering(img, 30, 30)

In [ ]:
# img_ms = cv2.pyrMeanShiftFiltering(img_ms, 30, 30)

In [ ]:
# 境界線の抽出
img_ms_gray = cv2.cvtColor(img_ms, cv2.COLOR_RGB2GRAY)
edge = cv2.bilateralFilter(img_gray, 9, 75, 75)
edge = cv2.Canny(edge, 50, 150, apertureSize=3)
edge = cv2.cvtColor(edge, cv2.COLOR_GRAY2RGB)

In [ ]:
img_dst = cv2.subtract(img_ms, edge)

In [ ]:
compare_plot([img, img_ms, edge, img_dst], \
             ["original", "mean_shift", "edge", "subtract"], \
            [None, None, "gray", "subtract"])

<a id="grabcut"></a>
## grabcut法

まず前景と背景の初期境界(rect)を大まかに指定し、次にそれぞれの色分布を基にグラフカットで前景と背景の境界を求めます。この新しい境界から求めた前景と背景の色分布計算と Graph Cut による境界算出を繰り返し行うことで、前景と背景を求めます。

アルゴリズムは数学的に扱うとかなり難しいのでここでは割愛しますが、このgrabcut法とマスクの手入力によって1枚における処理の時間を減らそうという手法です。

In [ ]:
plt.imshow(img)

画像の領域で見てみると
+ x軸はだいたい50から400前後
+ y軸はだいたい50から512まで

にLennaさんがいるのでこれらを対象に矩形を取っていきます。

In [ ]:
rect = (50, 50, 420, 512)

次にマスク用の領域を作っていきます。grabcutではあとから人間の手で前景と背景を選択できるので画像処理がうまく行かなかったときに修正するように作成していきます。

In [ ]:
mask = np.zeros(img.shape[:2], np.uint8)

In [ ]:
# 前景と背景のGMMモデルを作成します
bgModel = np.zeros((1, 65), np.float64)
fgModel = np.zeros((1, 65), np.float64)

In [ ]:
cv2.grabCut(img, mask, rect, bgModel, fgModel, 5, cv2.GC_INIT_WITH_RECT)
# maskの値が1の部分のみ残して0をクリアする
mask2 = np.where((mask == 2) | (mask == 0), 0,1).astype(np.uint8)
img_grab = img * mask2[:,:, np.newaxis]

In [ ]:
plt.imshow(img_grab)

帽子の一部が矩形として取得されなかったので、この部分を前景として修正していきましょう。

In [ ]:
from IPython.core.display import HTML
HTML('''
<canvas id="canvas" height="512px" width="512px" style="border: 1px solid;background: url('../../data/Lenna_flat.png')"></canvas>
<p>
    <button id="clear">クリア</button>
    <button id="white">前景(白色)</button>
    <button id="black">背景(黒色)</button>
    <button id="submit">画像のbase64化</button>
</p>
<p id="msg"></p>
<script>
    var kernel = IPython.notebook.kernel;

    var config = {
        "linesize": 12,
        "linecolor": "#000000"
    }

    var mouse = {
        "X": null,
        "Y": null,
    }

    var clear = document.getElementById("clear");
    var submit = document.getElementById("submit");
    var white = document.getElementById("white");
    var black = document.getElementById("black");
    var canvas = document.getElementById("canvas");
    var ctx = canvas.getContext("2d");

    ctx.strokeStyle = config.linecolor;
    
    clear.addEventListener("click", function(){
        ctx.clearRect(0, 0, canvas.width, canvas.height);
    });

    submit.addEventListener("click", function(){
        var variable_value = 'img_base64';
        kernel.execute(variable_value + " = '" + canvas.toDataURL() + "'");
        msg.textContent = "Success: " + "image -> " + variable_value;
    });
    
    white.addEventListener("click", function(){
        ctx.strokeStyle = "#FFFFFF";
    });
    
    black.addEventListener("click", function(){
        ctx.strokeStyle = "#000000";
    });

    canvas.addEventListener("mouseup", drawEnd, false);
    canvas.addEventListener("mouseout", drawEnd, false);
    
    canvas.addEventListener("mousemove", function(e){
        if (e.buttons === 1 || e.witch === 1) {
            var rect = e.target.getBoundingClientRect();
            var X = e.clientX - rect.left;
            var Y = e.clientY - rect.top;
            draw(X, Y);
        };
    });
 
    canvas.addEventListener("mousedown", function(e){
        if (e.button === 0) {
            var rect = e.target.getBoundingClientRect();
            var X = e.clientX - rect.left;
            var Y = e.clientY - rect.top;
            draw(X, Y);
        }
    });

    function draw(X, Y) {
        ctx.beginPath();
        if (mouse.X === null) {
            ctx.moveTo(X, Y);
        } else {
            ctx.moveTo(mouse.X, mouse.Y);
        }
        ctx.lineTo(X, Y);
        
        ctx.lineCap = "round";
        ctx.lineWidth = config.linesize * 2;
        
        ctx.stroke();

        mouse.X = X;
        mouse.Y = Y;
    };
 
    function drawEnd() {
        mouse.X = null;
        mouse.Y = null;
    }
</script>
''')

In [ ]:
from io import BytesIO
from PIL import Image
import base64

img_base64 = img_base64.split(",")[-1]
img_rect = np.array(Image.open(BytesIO(base64.b64decode(img_base64))))

In [ ]:
img_rect = cv2.cvtColor(img_rect, cv2.COLOR_RGBA2GRAY) // 255

In [ ]:
mask3 = cv2.bitwise_or(mask2, img_rect)

In [ ]:
compare_plot([img_rect, mask2, mask3], \
            ["manual_mask", "grabcut_mask", "bitwise_or"],
            ["gray" for _ in range(3)])

In [ ]:
cv2.grabCut(img, mask3, None, bgModel, fgModel, 5, cv2.GC_INIT_WITH_MASK)
mask = np.where((mask3 == 2) | (mask3 == 0), 0, 1).astype(np.uint8)

In [ ]:
img_grab2 = img * mask[:, :, np.newaxis]

In [ ]:
plt.imshow(img_grab2)

grabcut法と手入力によるマスクによってうまくLennaさんだけを抽出することができました！

<a id="conclusion"></a>
## まとめ
今回の画像処理ハンズオンにて紹介した処理は、画像処理の中のほんの少しでしかなく、これら以外にも様々な処理が存在しています。また最近のニューラルネットによる技術の向上によりGANなどによる画像生成など画像処理分野はより多種多様になっています。これをきっかけに画像処理分野により興味を持っていただけたらと思います。

## 参考資料・サイト
+ ディジタル画像処理 改訂新版 5,9,10章
+ Canny edge detection
> https://imagingsolution.net/imaging/canny-edge-detector/

+ バイラテラルフィルタ
> https://imagingsolution.net/imaging/bilateralfilter/

+ ノンローカルミーンフィルタ
> http://opencv.jp/opencv2-x-samples/non-local-means-filter

+ MeanShiftを用いたImage Segmentation
> https://news.mynavi.jp/article/cv_future-35/